In [9]:
#need rasterio to load GeoTIFF files and numpy to manipulate the data
import rasterio
import numpy as np
import os
from scipy.spatial import transform

In [104]:
# define satellite position in ECEF 

lat = 26.6
lon =-80.86
alt = 550e3
x = (alt + 6371e3) * np.cos(np.radians(lat)) * np.cos(np.radians(lon))
y = (alt + 6371e3) * np.cos(np.radians(lat)) * np.sin(np.radians(lon))
z = (alt + 6371e3) * np.sin(np.radians(lat))
print(x, y, z)

satellite_position = np.array([x, y, z])

1019200.3467890656 -5780172.397292891 3667571.2277580113


In [67]:
# define satellite orientation in ECEF

# pointing nadir in world coordinates
zc = dir_vector = -np.array([x, y, z]) / np.linalg.norm([x, y, z])
axis_of_rotation_z = np.cross(np.array([0,0,1]), dir_vector)
rc = axis_of_rotation_z = axis_of_rotation_z / np.linalg.norm(axis_of_rotation_z)
xc = -rc 

yc = south_vector = np.cross(rc, zc)
R = np.stack([-xc, yc, zc], axis=-1)
rot = transform.Rotation.from_matrix(R)
quaternion = rot.as_quat()
print(quaternion)


[-0.18880273 -0.37779677 -0.2688351   0.86565052]


In [68]:
# define camera
width = 640
height = 480
hfov = 66.1
half_angle = np.deg2rad(hfov / 2)
half_width = width/2
f = half_width / np.tan(half_angle)
resolution = np.array([width, height])

K = np.array([[f, 0, half_width], [0, f, height/2], [0, 0, 1]])

x = np.linspace(-half_width, half_width, width)
y = np.linspace(-height/2, height/2, height)
xx, yy = np.meshgrid(x, y)
ray_directions = np.stack([xx, yy, np.ones_like(xx)], axis=-1)
ray_directions /= np.linalg.norm(ray_directions, axis=-1, keepdims=True)

camera_orientation = R.T
ray_directions_ecef = ray_directions @ camera_orientation.T


In [103]:
def intersect_ellipsoid(ray_directions, satellite_position, a = 6378137.0, b = 6356752.314245):
    """
    Vectorized computation of ray intersections with the WGS84 ellipsoid.

    Parameters:
        ray_directions (np.ndarray): Array of ray directions (Nx3).
        satellite_position (np.ndarray): Satellite position in ECEF (3,).
        a (float): Semi-major axis of the WGS84 ellipsoid (meters).
        b (float): Semi-minor axis of the WGS84 ellipsoid (meters).

    Returns:
        np.ndarray: Intersection points (Nx3), or NaN for rays that miss.
    """

    ray_directions_flat = ray_directions.reshape(-1, 3)


    A = ray_directions_flat[:, 0]**2 / a**2 + ray_directions_flat[:, 1]**2 / a**2 + ray_directions_flat[:, 2]**2 / b**2
    B = 2 * (satellite_position[0] * ray_directions_flat[:, 0] / a**2 + 
                satellite_position[1] * ray_directions_flat[:, 1] / a**2 + 
                satellite_position[2] * ray_directions_flat[:, 2] / b**2)
    C = (satellite_position[0]**2 / a**2 +
            satellite_position[1]**2 / a**2 +
            satellite_position[2]**2 / b**2 - 1)
    discriminant = B**2 - 4 * A * C

    # Initialize intersection points as NaN
    intersection_points_flat = np.full_like(ray_directions_flat, np.nan)

    valid_mask = discriminant >= 0
    if np.any(valid_mask):
        # Compute roots of the quadratic equation
        sqrt_discriminant = np.sqrt(discriminant[valid_mask])
        t1 = (-B[valid_mask] - sqrt_discriminant) / (2 * A[valid_mask])
        t2 = (-B[valid_mask] + sqrt_discriminant) / (2 * A[valid_mask])

        # Choose the smallest positive t
        t = np.where((t1 > 0) & ((t1 < t2) | (t2 <= 0)), t1, t2)
        t = np.where(t > 0, t, np.nan)  # Filter out negative t values

        # Calculate intersection points
        valid_ray_directions = ray_directions_flat[valid_mask]
        intersection_points_flat[valid_mask] = t[:, None] * valid_ray_directions + satellite_position
    return intersection_points
intersection_points = intersect_ellipsoid(ray_directions_ecef, satellite_position)

def convert_to_lat_lon(intersection_points):
    """
    Convert intersection points (ECEF) to latitude and longitude.

    Parameters:
        intersection_points (np.ndarray): Array of intersection points (HxWx3) in ECEF coordinates.

    Returns:
        np.ndarray: Array of latitude and longitude (HxWx2), or NaN for invalid points.
    """

    H, W, _ = intersection_points.shape
    intersection_points_flat = intersection_points.reshape(-1, 3)

    valid_mask = ~np.isnan(intersection_points_flat).any(axis=1)
    
    lat_lon_flat = np.full((H * W, 2), np.nan)

    valid_points = intersection_points_flat[valid_mask]

    x, y, z = valid_points[:, 0], valid_points[:, 1], valid_points[:, 2]
    lat = np.degrees(np.arctan2(z, np.sqrt(x**2 + y**2)))
    lon = np.degrees(np.arctan2(y, x))

    lat_lon_flat[valid_mask, 0] = lat
    lat_lon_flat[valid_mask, 1] = lon

    return lat_lon_flat.reshape(H, W, 2)

lat_lon = convert_to_lat_lon(intersection_points)
print(lat_lon)



[[[34.59273242 46.82846187]
  [34.58893144 46.81972292]
  [34.58510571 46.81094734]
  ...
  [26.29218152 39.39870242]
  [26.28575288 39.39796773]
  [26.27935642 39.39724748]]

 [[34.5978309  46.840058  ]
  [34.59404701 46.83133122]
  [34.59023841 46.82256774]
  ...
  [26.28367018 39.39767019]
  [26.27725756 39.39695184]
  [26.27087713 39.39624785]]

 [[34.60292787 46.85168624]
  [34.59916118 46.84297184]
  [34.59536981 46.83422065]
  ...
  [26.2751446  39.3966554 ]
  [26.26874815 39.39595341]
  [26.2623839  39.39526571]]

 ...

 [[33.84675425 51.64349321]
  [33.84101205 51.64382427]
  [33.83523607 51.64414035]
  ...
  [24.56825328 43.01203598]
  [24.56471551 43.00335219]
  [24.56120578 42.99471085]]

 [[33.83900074 51.64398871]
  [33.83323745 51.64429851]
  [33.82744033 51.64459318]
  ...
  [24.57296044 43.02372441]
  [24.56940001 43.01502016]
  [24.5658677  43.00635833]]

 [[33.83125165 51.64445372]
  [33.82546739 51.64474225]
  [33.81964929 51.64501552]
  ...
  [24.57767804 43.035390